In [ ]:
import yaml
import pandas as pd
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))
from config import *

trace_list = []
with open('../../traces/trace_combinations.txt', "r") as f:
  for line in f:
    trace_list.append(line.strip().split(','))

data_rows = []
for trace_group in trace_list:
  for device in device_list:
    for org in org_list[device]:
      for timing in timing_list[device]:
        for row_policy in row_policy_list[device]:
          for refresh_manager in refresh_manager_list[device]:
            for area_scale in area_scaling_list[device]:
              for voltage_scale in voltage_scaling_list[device]:
                run_file = f'{device}_{org}_{timing}_{row_policy}_{refresh_manager}_{area_scale}_{voltage_scale}_{trace_group[0]}'
                with open(f'../../results/stats/{run_file}.yaml', 'r') as f:
                  data_rows.append({
                    'Device': device,
                    'Org': org,
                    'Timing': timing,
                    'Row Policy': row_policy,
                    'Refresh Manager': refresh_manager,
                    'Area Scaling': area_scale,
                    'Voltage Scaling': voltage_scale,
                    'Trace': trace_group[0],
                    'Metric': 0.0,
                    'Yaml': yaml.safe_load(f)
                  })

df = pd.DataFrame(data_rows)
sc_df = df[~df['Trace'].str.startswith(tuple(['H', 'L']))] #singlecore run results
mc_df = df[df['Trace'].str.startswith(tuple(['H', 'L']))]  #multicore run results

### Singelcore Plots

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

color_palette =['#E78F81', '#4A4947']

sc_df.loc[:, 'Metric'] = sc_df['Yaml'].apply(lambda x: x['Frontend']['num_expected_insts'] / x['Frontend']['cycles_recorded_core_0'])
uram5_df = sc_df[(sc_df['Device'] == 'URAM5') & (sc_df['Row Policy'] == 'OpenRowPolicy')]
ddr5_df  = sc_df[(sc_df['Device'] == 'DDR5' ) & (sc_df['Row Policy'] == 'OpenRowPolicy')]
for area_scale in area_scaling_list['URAM5']:
  combined_df = pd.concat([uram5_df[(uram5_df['Area Scaling'] == area_scale) & (uram5_df['Voltage Scaling'] == 1.0)], ddr5_df])

  avg_df = combined_df.groupby('Device')['Metric'].mean().reset_index()
  avg_df['Trace'] = 'Average'
  combined_df = pd.concat([combined_df, avg_df])

  plt.figure(figsize=(16, 6))
  barplot = sns.barplot(data=combined_df, x='Trace', y='Metric', hue='Device',
                        edgecolor='black', linewidth=1.5,
                        zorder=2, palette=color_palette)

  barplot.set_xlabel('Benchmark', fontsize=12)
  barplot.set_ylabel('IPC', fontsize=12)
  barplot.set_title(f'Area Scaling {area_scale}', fontsize=12)
  barplot.grid(True, which='major', axis='y', linestyle='--', linewidth=1, color='gray', zorder=0)
  barplot.legend(loc='upper right')

  plt.xticks(rotation=90)
  plt.axvline(x=len(combined_df['Trace'].unique()) - 1 - 0.5, color='black', linestyle='--', linewidth=1.5)

  plt.tight_layout()
  plt.show()

### Multicore Plots

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

color_palette =['#629584', '#4A4947']
trace_groups = {trace[0]: trace[1:] for trace in trace_list if trace[0].startswith(('H', 'L', 'M'))}

for index, rows in mc_df.iterrows():
  weighted_ipc = 0.0
  mc_trace_ipc = 0.0
  for idx, trace in enumerate(trace_groups[rows['Trace']]):
    conditions = (
      (sc_df['Device'] == 'DDR5') &
      (sc_df['Org'] == rows['Org']) &
      (sc_df['Timing'] == rows['Timing']) &
      (sc_df['Row Policy'] == rows['Row Policy']) &
      (sc_df['Refresh Manager'] == 'AllBank') &
      (sc_df['Voltage Scaling'] == 1.0) &
      (sc_df['Area Scaling'] == 1.0) &
      (sc_df['Trace'] == trace)
    )
    mc_trace_ipc = rows['Yaml']['Frontend']['num_expected_insts'] / rows['Yaml']['Frontend'][f'cycles_recorded_core_{idx}']
    sc_trace_ipc = sc_df[conditions]['Yaml'].item()['Frontend']['num_expected_insts'] / sc_df[conditions]['Yaml'].item()['Frontend']['cycles_recorded_core_0']
    weighted_ipc += mc_trace_ipc / sc_trace_ipc
  mc_df.at[index, 'Metric'] = weighted_ipc

# ------------------------------------------------------------------------------
# ------------------------------------------------------------------------------
# ------------------------------------------------------------------------------

uram5_df = mc_df[(mc_df['Device'] == 'URAM5') & (mc_df['Row Policy'] == 'OpenRowPolicy')]
ddr5_df  = mc_df[(mc_df['Device'] == 'DDR5' ) & (mc_df['Row Policy'] == 'OpenRowPolicy')]
for area_scale in area_scaling_list['URAM5']:
  combined_df = pd.concat([uram5_df[(uram5_df['Area Scaling'] == area_scale) & (uram5_df['Voltage Scaling'] == 1.0)], ddr5_df])

  avg_df = combined_df.groupby('Device')['Metric'].mean().reset_index()
  avg_df['Trace'] = 'Average'
  combined_df = pd.concat([combined_df, avg_df])

  plt.figure(figsize=(16, 6))
  barplot = sns.barplot(data=combined_df, x='Trace', y='Metric', hue='Device',
                        edgecolor='black', linewidth=1.5,
                        zorder=2, palette=color_palette)

  barplot.set_xlabel('Benchmark', fontsize=12)
  barplot.set_ylabel('System Throughput (Weighted Speedup)', fontsize=12)
  barplot.set_title(f'Area Scaling {area_scale}', fontsize=12)
  barplot.grid(True, which='major', axis='y', linestyle='--', linewidth=1, color='gray', zorder=0)
  barplot.legend(loc='upper right')

  plt.xticks(rotation=90)
  plt.axvline(x=len(combined_df['Trace'].unique()) - 1 - 0.5, color='black', linestyle='--', linewidth=1.5)

  plt.tight_layout()
  plt.show()

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

color_palette =['#BB9AB1', '#4A4947']
trace_groups = {trace[0]: trace[1:] for trace in trace_list if trace[0].startswith(('H', 'L', 'M'))}

#TODO: looks like shit
for index, rows in mc_df.iterrows():
  weighted_ipc = 0.0
  mc_trace_ipc = 0.0
  for idx, trace in enumerate(trace_groups[rows['Trace']]):
    conditions = (
      (sc_df['Device'] == rows['Device']) &
      (sc_df['Org'] == rows['Org']) &
      (sc_df['Timing'] == rows['Timing']) &
      (sc_df['Row Policy'] == rows['Row Policy']) &
      (sc_df['Refresh Manager'] == rows['Refresh Manager']) &
      (sc_df['Area Scaling'] == rows['Area Scaling']) &
      (sc_df['Trace'] == trace)
    )
    mc_trace_ipc = rows['Yaml']['Frontend']['num_expected_insts'] / rows['Yaml']['Frontend'][f'cycles_recorded_core_{idx}']
    sc_trace_ipc = sc_df[conditions]['Yaml'].item()['Frontend']['num_expected_insts'] / sc_df[conditions]['Yaml'].item()['Frontend']['cycles_recorded_core_0']
    weighted_ipc += 1 / (mc_trace_ipc / sc_trace_ipc)
  mc_df.at[index, 'Metric'] = 1 / (4 / weighted_ipc)  # antt is the reciprocal of harmonic mean

# ------------------------------------------------------------------------------
# ------------------------------------------------------------------------------
# ------------------------------------------------------------------------------

uram5_df = mc_df[(mc_df['Device'] == 'URAM5') & (mc_df['Row Policy'] == 'OpenRowPolicy')]
ddr5_df  = mc_df[(mc_df['Device'] == 'DDR5' ) & (mc_df['Row Policy'] == 'OpenRowPolicy')]
# for area_scaling in scale_list['URAM5']:
for area_scaling in [1.0]:
  combined_df = pd.concat([uram5_df[uram5_df['Area Scaling'] == area_scaling], ddr5_df])

  avg_df = combined_df.groupby('Device')['Metric'].mean().reset_index()
  avg_df['Trace'] = 'Average'
  combined_df = pd.concat([combined_df, avg_df])

  plt.figure(figsize=(16, 6))
  barplot = sns.barplot(data=combined_df, x='Trace', y='Metric', hue='Device',
                        edgecolor='black', linewidth=1.5,
                        zorder=2, palette=color_palette)

  barplot.set_xlabel('Benchmark', fontsize=12)
  barplot.set_ylabel('Average Normalized Turnaround Time (ANTT)', fontsize=12)
  barplot.set_title(f'Area Scaling {area_scaling}', fontsize=12)
  barplot.grid(True, which='major', axis='y', linestyle='--', linewidth=1, color='gray', zorder=0)
  barplot.legend(loc='upper right')

  plt.xticks(rotation=90)
  plt.axvline(x=len(combined_df['Trace'].unique()) - 1 - 0.5, color='black', linestyle='--', linewidth=1.5)

  plt.tight_layout()
  plt.show()